In [ ]:
!pip install --upgrade tensorflow
!pip install tf-agents

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Replace 'your_dataset.csv' with the actual file path
df = pd.read_csv('/content/WEKA_fileiras_nao_sorteadas.csv')

In [ ]:
df.columns

Index(['fileiras_nao_sorteadas_str', 'P_2n_7_str', 'dz1_dz6_dist_fileiras_int',
       'existe_zero_a_esquerda_fileira_medianas_str',
       'existe_zero_a_direita_fileira_medianas_str',
       'dist_entre_fileiras_com_zero_int', 'qtde_fileiras_sorteadas',
       'dz1_dz6_soma_int'],
      dtype='object')

In [ ]:
# Replace 'your_dataset.csv' with the actual file path
df = pd.read_csv('/content/WEKA_fileiras_nao_sorteadas.csv')

y = df['fileiras_nao_sorteadas_str']  # Target variable
X = df.drop('fileiras_nao_sorteadas_str', axis=1)  # Features

In [ ]:
# Replace 'your_dataset.csv' with the actual file path
df = pd.read_csv('/content/WEKA_fileiras_nao_sorteadas.csv')

y = df['fileiras_nao_sorteadas_str']  # Target variable
X = df.drop('fileiras_nao_sorteadas_str', axis=1)  # Features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Define numerical and categorical features
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Create transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline with the preprocessor and the classifier
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('classifier', RandomForestClassifier())])


pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9774859287054409


In [ ]:
import gym
from gym import spaces
import numpy as np

class CustomDatasetEnv(gym.Env):
    def __init__(self, dataset):
        super(CustomDatasetEnv, self).__init__()

        # Assuming the dataset is a Pandas DataFrame
        self.dataset = dataset
        self.current_index = 0  # Track the current index in the dataset

        # Define observation space based on the number of features in your dataset
        self.observation_space = spaces.Box(low=0, high=1, shape=(len(X.columns),), dtype=np.float32)

        # Define action space (if applicable)
        self.action_space = spaces.Discrete(len(y))  # Replace num_actions with the actual number of actions

    def reset(self):
        # Reset the environment to the initial state
        self.current_index = 0
        initial_observation = self.dataset.iloc[self.current_index].values
        return initial_observation

    def step(self, action):
        # Take an action in the environment
        # Update the state based on the dataset and the action
        # Calculate the reward based on your task
        # Return the next observation, reward, done flag, and additional information

        # Example:
        self.current_index += 1
        if self.current_index < len(self.dataset):
            next_observation = self.dataset.iloc[self.current_index].values
            reward = calculate_reward()  # Implement your own reward function
            done = False
        else:
            next_observation = np.zeros_like(self.observation_space.low)  # or any other appropriate value
            reward = 0
            done = True

        return next_observation, reward, done, {}

# Register the custom environment with OpenAI Gym
gym.envs.register(
    id='CustomDataset-v0',
    entry_point='env:CustomDatasetEnv',  # Replace 'your_module' with the actual module containing your environment
)

# Create an instance of the custom environment
env = gym.make('CustomDataset-v0')



/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:595: UserWarning: WARN: Overriding environment CustomDataset-v0
  logger.warn(f"Overriding environment {id}")


ModuleNotFoundError: ignored

In [ ]:
import tensorflow as tf
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.environments import tf_py_environment
from tf_agents.environments import suite_gym

# Convert your custom environment to TensorFlow environment
tf_env = tf_py_environment.TFPyEnvironment(suite_gym.load('CustomDataset-v0'))

# Define the Q-network
fc_layer_params = (100, )
q_net = q_network.QNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    fc_layer_params=fc_layer_params)

# Define the DQN agent
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3)
train_step_counter = tf.Variable(0)
agent = dqn_agent.DqnAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

# Define the training loop
num_iterations = 10000

for _ in range(num_iterations):
    # Collect a trajectory from the environment
    trajectory = agent.collect_policy.collect(
        tf_env, agent.collect_policy.get_initial_state(tf_env.batch_size))

    # Update the agent's policy
    experience, _ = utils.batch_tf_env(tf_env)(trajectory)
    train_loss = agent.train(experience)

    # Other logging or monitoring logic


NameNotFound: ignored

In [ ]:
# Define the evaluation loop
num_eval_episodes = 10

for _ in range(num_eval_episodes):
    # Collect a trajectory from the environment using the trained policy
    trajectory = agent.policy.collect_policy.collect(
        tf_env, agent.policy.get_initial_state(tf_env.batch_size))

    # Other evaluation logic
